# AIM: 이미지 평가 v2 - CLIP 히트맵 & 캡션
- 주제(topic), 프롬프트(prompt), 이미지들을 업로드하고 기준 이미지 선택
- **NEW**: CLIP 어텐션 히트맵으로 이미지-텍스트 유사 영역 시각화
- **NEW**: BLIP2가 생성한 캡션을 결과 테이블에 표시
- Hugging Face 토큰은 `.env`의 `HUGGING_FACE_API` 또는 `HF_TOKEN` 사용

In [1]:
# 환경 준비: 프로젝트 루트 경로 추가 및 .env 로딩
import os, sys
from dotenv import load_dotenv, find_dotenv
ROOT = os.path.dirname(os.path.dirname(os.getcwd())) if os.path.basename(os.getcwd())=='notebooks' else os.getcwd()
if ROOT not in sys.path: sys.path.insert(0, ROOT)
load_dotenv(find_dotenv())
print('ROOT =', ROOT)

ROOT = /abr/co_show14/aim_eval


In [2]:
# 위젯 UI 및 썸네일 프리뷰
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from PIL import Image as PILImage
import matplotlib.pyplot as plt
import numpy as np
import cv2

# 입력창 크기 확대
topic_w = widgets.Textarea(
    value='A wooden cabin with a smoking chimney stands among snow-covered trees and a snowy mountain in the back.',
    description='Topic',
    style={'description_width':'initial'},
    layout=widgets.Layout(width='800px', height='80px')
)

prompt_w = widgets.Textarea(
    value='A cozy wooden cabin with a smoking chimney in a snowy forest, majestic snow-covered mountain in the background, crisp winter day, cinematic lighting, hyperrealistic, 8k',
    description='Prompt',
    style={'description_width':'initial'},
    layout=widgets.Layout(width='800px', height='80px')
)

files_w = widgets.FileUpload(accept='image/*', multiple=True, description='Upload Images')
ref_dropdown = widgets.Dropdown(options=[], description='Reference Image', style={'description_width':'initial'})
enable_clip_w = widgets.Checkbox(value=True, description='Enable CLIP')
enable_blip2_w = widgets.Checkbox(value=True, description='Enable BLIP2')
enable_lpips_w = widgets.Checkbox(value=True, description='Enable LPIPS')
run_btn = widgets.Button(description='Run Evaluation', button_style='primary')
thumbs_out = widgets.Output()
out = widgets.Output()
heatmap_out = widgets.Output()  # CLIP 히트맵 출력

In [3]:
# CLIP 히트맵 생성 함수
import torch

def create_clip_heatmap(eval_inst, image_path, prompt):
    """CLIP 어텐션 맵을 생성하여 이미지-텍스트 유사 영역을 시각화"""
    if not eval_inst.enable_clip:
        return None
        
    from PIL import Image
    import matplotlib.pyplot as plt
    
    # 이미지 로드 및 전처리
    image = Image.open(image_path).convert('RGB')
    inputs = eval_inst.clip_processor(
        text=[prompt],
        images=image,
        return_tensors='pt',
        padding=True
    )
    inputs = {k: v.to(eval_inst.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        # CLIP 비전 인코더에서 어텐션 가중치 추출
        vision_outputs = eval_inst.clip_model.vision_model(**{'pixel_values': inputs['pixel_values']}, output_attentions=True)
        text_outputs = eval_inst.clip_model.text_model(**{'input_ids': inputs['input_ids'], 'attention_mask': inputs.get('attention_mask')})
        
        # 마지막 레이어의 어텐션 가중치 사용
        attention_weights = vision_outputs.attentions[-1]  # [batch, heads, seq_len, seq_len]
        
        # CLS 토큰(첫 번째)에 대한 어텐션을 평균화
        cls_attention = attention_weights[0, :, 0, 1:].mean(0)  # [patches]
        
        # 패치 그리드로 재구성 (ViT-B/32는 7x7 패치)
        patch_size = int(cls_attention.shape[0] ** 0.5)
        attention_map = cls_attention.reshape(patch_size, patch_size)
        attention_map = attention_map.cpu().numpy()
        
        # 원본 이미지 크기로 업스케일
        attention_resized = cv2.resize(attention_map, image.size, interpolation=cv2.INTER_CUBIC)
        
        return image, attention_resized

def display_clip_heatmap(image, attention_map, title):
    """CLIP attention heatmap visualization"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(image)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # Attention map
    im = axes[1].imshow(attention_map, cmap='hot', alpha=0.8)
    axes[1].set_title('CLIP Attention Map')
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Overlay
    axes[2].imshow(image)
    axes[2].imshow(attention_map, cmap='hot', alpha=0.4)
    axes[2].set_title('Overlay (Focus Areas)')
    axes[2].axis('off')
    
    plt.suptitle(f'{title} - CLIP Attention Analysis', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

print('CLIP heatmap functions ready')

CLIP heatmap functions ready


In [4]:
# 평가 실행 로직
import tempfile, io
from PIL import Image
import pandas as pd
import evaluation

def run_eval_clicked(_):
    with out:
        clear_output()
        if not files_w.value:
            print('이미지를 업로드하세요.')
            return
        
        # 진행 상태 표시
        print("모델 로딩 중...")
        
        # 환경 플래그 설정
        os.environ['DISABLE_CLIP'] = '0' if enable_clip_w.value else '1'
        os.environ['DISABLE_BLIP2'] = '0' if enable_blip2_w.value else '1'
        os.environ['DISABLE_LPIPS'] = '0' if enable_lpips_w.value else '1'
        
        # 평가기 인스턴스
        eval_inst = evaluation.get_evaluator()
        
        print("이미지 처리 중...")
        
        # 업로드 파일을 임시경로에 저장
        tmpdir = tempfile.mkdtemp(prefix='aim_nb_v2_')
        name_to_path = {}
        for i, item in enumerate(files_w.value):
            if hasattr(item, 'name') and item.name:
                name = item.name
            else:
                meta = item.get('metadata') or {}
                name = meta.get('name', f'image_{i+1}.png')
            
            content = item['content'] if isinstance(item['content'], (bytes, bytearray)) else item['content'].tobytes()
            img = Image.open(io.BytesIO(content)).convert('RGB')
            path = os.path.join(tmpdir, name)
            img.save(path)
            name_to_path[name] = path
            
        ref_name = ref_dropdown.value
        if ref_name not in name_to_path:
            print('기준 이미지가 목록에 없습니다.')
            return
            
        print("이미지 평가 중...")
        
        topic = topic_w.value
        prompt = prompt_w.value
        results = []
        captions = {}  # 캡션을 별도로 저장
        
        # 기준 이미지는 LPIPS를 제외하고 평가
        for name, path in name_to_path.items():
            if name == ref_name:
                # 기준 이미지는 LPIPS 제외
                r = eval_inst.evaluate(path, name_to_path[ref_name], prompt, topic)
                r['lpips_score'] = None
                r['lpips_norm'] = None
                # 기준 이미지는 LPIPS 제외하고 final_score 재계산
                clip_norm = r['clip_norm']
                blip2_norm = r['blip2_norm']
                weights = {
                    'clip': 0.2 if enable_clip_w.value else 0.0,
                    'blip2': 0.5 if enable_blip2_w.value else 0.0,
                }
                weight_sum = sum(weights.values()) or 1.0
                score = (clip_norm * weights['clip'] + blip2_norm * weights['blip2'])
                r['final_score'] = (score / weight_sum) * 100
                r['is_reference'] = True
            else:
                # 다른 이미지는 전체 평가
                r = eval_inst.evaluate(path, name_to_path[ref_name], prompt, topic)
                r['is_reference'] = False
            
            # 생성된 캡션을 별도로 저장 (BLIP2가 활성화된 경우)
            if enable_blip2_w.value and hasattr(eval_inst, '_caption_cache'):
                captions[name] = eval_inst._caption_cache.get(path, '캡션 생성 실패')
            
            r['filename'] = name
            results.append(r)
            
        if not results:
            print('평가 결과가 없습니다.')
            return
            
        print("평가 완료")
        
        # 캡션을 제외한 결과 테이블 표시
        columns = ['filename', 'clip_score', 'clip_norm', 'blip2_similarity', 'blip2_norm', 'lpips_score', 'lpips_norm', 'final_score']
        df = pd.DataFrame(results)[columns]
        display(df)
        
        # 생성된 캡션을 별도로 표시
        if enable_blip2_w.value and captions:
            print("\n=== Generated Captions ===")
            for name, caption in captions.items():
                print(f"{name}: {caption}")
        
        # CLIP 히트맵 생성 및 표시
        if enable_clip_w.value:
            print("CLIP 어텐션 히트맵 생성 중...")
            with heatmap_out:
                clear_output()
                for result in results:
                    name = result['filename']
                    path = name_to_path[name]
                    try:
                        heatmap_result = create_clip_heatmap(eval_inst, path, prompt)
                        if heatmap_result:
                            image, attention_map = heatmap_result
                            display_clip_heatmap(image, attention_map, name)
                    except Exception as e:
                        print(f"{name} CLIP 히트맵 생성 실패: {e}")
            print("CLIP 어텐션 히트맵 완료!")
       
run_btn.on_click(run_eval_clicked)
print('Ready for v2 evaluation.')

Ready for v2 evaluation.


In [5]:
def render_thumbnails(change=None):
    with thumbs_out:
        clear_output()
        if not files_w.value:
            print('이미지를 업로드하세요.')
            return
        items = []
        current_ref = ref_dropdown.value or ''
        for i, f in enumerate(files_w.value):
            # 파일명 추출 개선
            if hasattr(f, 'name') and f.name:
                name = f.name
            else:
                meta = f.get('metadata') or {}
                name = meta.get('name', f'image_{i+1}.png')
            
            content = f['content'] if isinstance(f['content'], (bytes, bytearray)) else f['content'].tobytes()
            img = PILImage.open(io.BytesIO(content)).convert('RGB')
            thumb = img.copy()
            thumb.thumbnail((160,160))
            buf = io.BytesIO()
            thumb.save(buf, format='PNG')
            wimg = widgets.Image(value=buf.getvalue(), format='png', width=160, height=160)
            border = '3px solid #0056b3' if name == current_ref else '1px solid #ccc'
            box = widgets.VBox([wimg, widgets.HTML(f"<div style='text-align:center; font-size:12px'>{name}</div>")],
                               layout=widgets.Layout(border=border, padding='4px', margin='4px', align_items='center', width='180px'))
            items.append(box)
        grid = widgets.GridBox(children=items, layout=widgets.Layout(grid_template_columns='repeat(4, 180px)'))
        display(grid)

def update_ref_options(change=None):
    names = []
    for i, f in enumerate(files_w.value):
        # 파일명 추출 개선
        if hasattr(f, 'name') and f.name:
            name = f.name
        else:
            meta = f.get('metadata') or {}
            name = meta.get('name', f'image_{i+1}.png')
        names.append(name)
    
    ref_dropdown.options = names
    if names: 
        ref_dropdown.value = names[0]
    render_thumbnails()

files_w.observe(update_ref_options, names='value')
ref_dropdown.observe(render_thumbnails, names='value')

# 평가 방식 설명 업데이트 (v2)
info_panel = widgets.HTML(
    value="""
    <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; border: 1px solid #dee2e6; margin-top: 10px;'>
        <h4 style='margin-top: 0; color: #495057;'>📊 AIM v2 평가 방식 설명</h4>
        <ul style='margin-bottom: 0; color: #6c757d; line-height: 1.6;'>
            <li><strong>🔗 CLIP 점수:</strong> 텍스트-이미지 유사도 (0.4 이상 → 1.0, 아니면 (score+1)/1.4)</li>
            <li><strong>💬 BLIP2 점수:</strong> 토픽 유사도 (0.7 이상 → 1.0, 아니면 score/0.7)</li>
            <li><strong>👁️ LPIPS 점수:</strong> 이미지 시각적 유사도 (낮을수록 좋음, 1-score로 정규화)</li>
            <li><strong>🏆 Final Score:</strong> 가중평균 (CLIP:20%, BLIP2:50%, LPIPS:30%) × 100</li>
        </ul>
        <div style='background-color: #e3f2fd; padding: 10px; border-radius: 6px; margin-top: 10px;'>
            <strong>🆕 v2 새로운 기능:</strong><br>
            • 테이블에 생성된 캡션 표시<br>  
            • CLIP 어텐션 히트맵으로 프롬프트와 유사한 이미지 영역 시각화<br>
            • 빨간색일수록 프롬프트와 유사하다고 CLIP이 판단한 영역
        </div>
    </div>
    """,
    layout=widgets.Layout(width='800px')
)

# UI 표시
display(
    topic_w, 
    prompt_w, 
    files_w, 
    ref_dropdown, 
    widgets.HBox([enable_clip_w, enable_blip2_w, enable_lpips_w]), 
    thumbs_out, 
    run_btn, 
    info_panel,
    out,
    widgets.HTML("<h2 style='color:#1976d2; margin-top:30px;'>🔥 CLIP 어텐션 히트맵</h2>"),
    heatmap_out  # CLIP 히트맵 출력 영역
)

Textarea(value='A wooden cabin with a smoking chimney stands among snow-covered trees and a snowy mountain in …

Textarea(value='A cozy wooden cabin with a smoking chimney in a snowy forest, majestic snow-covered mountain i…

FileUpload(value=(), accept='image/*', description='Upload Images', multiple=True)

Dropdown(description='Reference Image', options=(), style=DescriptionStyle(description_width='initial'), value…

Output()

Button(button_style='primary', description='Run Evaluation', style=ButtonStyle())

HTML(value="\n    <div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; border: 1px solid …

Output()

HTML(value="<h2 style='color:#1976d2; margin-top:30px;'>🔥 CLIP 어텐션 히트맵</h2>")

Output()